In [27]:
import pandas as pd
import datetime

In [112]:
df_users = pd.read_csv("/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4012/Data/combined_twitter_data_with_tweets_corpus.csv")

In [9]:
# get tweets df
base = "/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4012/"
filenames_tweets = [
    "Data/all tweets 2017/tweets_fake_followers.csv",
    "Data/all tweets 2017/tweets_genuine_accounts.csv",
    "Data/all tweets 2017/tweets_social_spambots_1.csv",
    "Data/all tweets 2017/tweets_social_spambots_2.csv",
    "Data/all tweets 2017/tweets_social_spambots_3.csv",
    "Data/all tweets 2017/tweets_traditional_spambots_1.csv",

    "Data/tweets 2015/tweets_E13.csv",
    "Data/tweets 2015/tweets_FSF.csv",
    "Data/tweets 2015/tweets_INT.csv",
    "Data/tweets 2015/tweets_TFP.csv",
    "Data/tweets 2015/tweets_TWT.csv"
]
filenames_tweets = map(lambda x: base+x, filenames_tweets)

for i,fn in enumerate(filenames_tweets):
    if i == 0:
        df_tweets = pd.read_csv(fn, encoding='ISO-8859-1')
    else:
        df_tweets = pd.concat([df_tweets, pd.read_csv(fn, encoding='ISO-8859-1') ], axis=0)


/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/1643796394.py:21: DtypeWarning: Columns (8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.read_csv(fn, encoding='ISO-8859-1')
/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/1643796394.py:23: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.concat([df_tweets, pd.read_csv(fn, encoding='ISO-8859-1') ], axis=0)
/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/1643796394.py:23: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.concat([df_tweets, pd.read_csv(fn, encoding='ISO-8859-1') ], axis=0)
/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/1643796394.py:23: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.concat([df_tweets, pd

In [15]:
df_tweets = df_tweets.dropna(subset = ["user_id"])  
df_tweets["user_id"] = df_tweets["user_id"].apply(int)

/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/1418784699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets["user_id"] = df_tweets["user_id"].apply(int)


## Date Formatting

In [39]:
# takes around 10 min to run
df_tweets['created_at_formatted'] = pd.to_datetime(df_tweets['timestamp'], infer_datetime_format=True, errors='coerce')

/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/2158826432.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['created_at_formatted'] = pd.to_datetime(df_tweets['timestamp'], infer_datetime_format=True, errors='coerce')


## User Tweet Frequency

In [105]:
# user tweet frequency = total number of tweets / number of user active days 
# shows how often the user tweets among the days that a user tweets at least once. User activity is defined by whether the user tweets in a given day
# 1 = user tweets only once per active day 
# >1 = user tweets more than once a day on average, in the days that the user is active 

df_tweets_per_day = df_tweets.groupby(by=["user_id"]).agg(tweet_count=('text', 'count'),
                                                          date_count=('created_at_formatted', lambda x: x.nunique()))

dict_tweets_average = {user_id: df_tweets_per_day.loc[user_id]['tweet_count'] / df_tweets_per_day.loc[user_id]['date_count'] for user_id in df_tweets_per_day.index}

/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/2468365992.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['tweet_placeholder'] = 1


In [114]:
#create new column for user tweet frequency 
df_users['tweet_frequency'] = df_users['id'].map(dict_tweets_average)

## Tags and Mentions

In [119]:
# average number of tags per post = total number of tags used per tweet 
# average number of mentions per post = total number of mentions per tweet 

df_tweets['text'] = df_tweets['text'].apply(str) #convert all text to string
df_tweets['number_of_tags'] = df_tweets['text'].apply(lambda x: x.count("#"))
df_tweets['number_of_mentions'] = df_tweets['text'].apply(lambda x: x.count("@"))

/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/1139840106.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['text'] = df_tweets['text'].apply(str) #convert all text to string
/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/1139840106.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['number_of_tags'] = df_tweets['text'].apply(lambda x: x.count("#"))
/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/1139840106.py:6: SettingWithCopyWarn

In [125]:
tags_dict = df_tweets.groupby(by=["user_id"])['number_of_tags'].sum().to_dict()
mentions_dict = df_tweets.groupby(by=["user_id"])['number_of_mentions'].sum().to_dict() 

#create new column for number of tags
df_users['number_of_tags'] = df_users['id'].map(tags_dict)
#create new column for number of mentions
df_users['number_of_mentions'] = df_users['id'].map(mentions_dict)


In [ ]:
# return 0 if weekend, 1 if weekday 
def is_weekday(dt):
    return 0 if dt.weekday() > 4 else 1

# return day of week 
def get_weekday(dt):
    return dt.weekday()